In [2]:
from pystac_client import Client

def query_stac(bbox, datetime, cloudcover):
    catalog = Client.open("https://cmr.earthdata.nasa.gov/stac/LPCLOUD")
    
    search = catalog.search(
        collections=["HLSL30.v2.0"],
        bbox=[-122.4969372889167, 59.282959601314104, -120.20175133769581, 60.40424594544261],
        datetime=['2020-06-01T00:00:00Z','2021-09-15T23:59:59Z'],
        # max_items=80, # for testing
        # query={"eo:cloud_cover":{"lt":20}} #doesn't work
    )
    results = search.get_all_items_as_dict()
    
    filtered_results = []
    for i in results['features']:
        if int(i['properties']['eo:cloud_cover']) <= cloudcover:
            filtered_results.append(i)
    
    results['features'] = filtered_results
    
    return results

results =query_stac([-122.4969372889167, 59.282959601314104, -120.20175133769581, 60.40424594544261], ['2020-06-01T00:00:00Z','2021-09-15T23:59:59Z'], 20)

In [3]:
results

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'HLS.L30.T10VFL.2020170T190406.v2.0',
   'stac_version': '1.0.0',
   'stac_extensions': ['https://stac-extensions.github.io/eo/v1.0.0/schema.json'],
   'collection': 'HLSL30.v2.0',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-119.3990656, 58.5017681],
      [-119.294711, 59.48584],
      [-120.666852, 59.5175446],
      [-121.230616, 58.5400954],
      [-119.3990656, 58.5017681]]]},
   'bbox': [-121.230616, 58.501768, -119.294711, 59.517545],
   'links': [{'rel': 'self',
     'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0/items/HLS.L30.T10VFL.2020170T190406.v2.0'},
    {'rel': 'parent',
     'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0'},
    {'rel': 'collection',
     'href': 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/HLSL30.v2.0'},
    {'rel': 'root', 'href': 'https://cmr.earthdata.nasa.gov/stac/'},
    {'rel': 'provider', '